# BB84 Key Distribution Protocol

In Part 1 of this assignment, you learnt about a simple quantum key distribution protocol (called BB84, after  C. H. Bennett and G. Brassard) for securely generating random bitstrings which can then be used to encrypt information to securely share it between parties. Implement this protocol in Q# using the template given below. 

#### Run these cells before beginning to import necessary packages.

In [1]:
import qsharp
import qsharp.azure
import matplotlib.pyplot as plt

Preparing Q# environment...


In [2]:
%%qsharp 
open Microsoft.Quantum.Diagnostics; 
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Random;

### Task 1: Random Bits
Implement an operation that returns a random bit. This will be used in further operations.

**Summary**

Returns a random bit (0 or 1) with equal probability.

**Output**

A random integer in \{0, 1\}

In [3]:
%%qsharp
operation RandomBit() : Int {
    return DrawRandomInt(0,1);
}

### Task 2: Alice (Sender)
The first step of this protocol is for Alice to generate a random bitstring and to encode a qubit string by initializing them randomly to $|0\rangle, |1\rangle, |+\rangle \text{ or } |-\rangle$. 

Note that $|0\rangle$ and $|1\rangle$ are the basis vectors in the Z basis and $|+\rangle$ and $|-\rangle$ are the basis vectors in the X basis. Therefore, we can say that Alice is encoding the random bitstring $S_A$ either in the Z basis ($|0\rangle$ and $|1\rangle$) or the X basis ($|+\rangle$ and $|-\rangle$).


**Summary**

This operation prepares each of the qubits in the input array in one of the |0⟩, |1⟩, |+⟩, or |-⟩ states randomly.
Additionally, it returns a tuple of two arrays, an array of the chosen bases (X or Z) and an array of the encoded bits (0 or 1).

**Input**

*qs*: A qubit array that the sender prepares randomly in X and Z basis as either 0 or 1

*Output*: A tuple of two integer arrays. Each element of the first array indicates the basis chosen, 0 for Z, 1 for X.
The corresponding element of the second array indicates whether the qubit was prepared as 0 or 1 in that basis.


In [4]:
%%qsharp
operation Alice(qs : Qubit[]) : (Int[], Int[]) {
    mutable bases = [];
    mutable values = [];
    for q in qs{
        if(RandomBit() == 1){
            if(RandomBit() == 1){
                //x basis, value 1 |->
                Reset(q);
                X(q);
                H(q);
                set bases += [1];
                set values += [1];

            }
            else{
                //x basis, value 0 |+>
                Reset(q);
                H(q);
                set bases += [1];
                set values += [0];
            }
        }
        else{
            if(RandomBit() == 1){
                //z-basis value 1 |1>
                Reset(q);
                X(q);
                set bases += [0];
                set values += [1];
            }
            else{
                //z-basis value 0 |0>
                Reset(q);
                set bases += [0];
                set values += [0];
            }
        }
    }
    return (bases, values);

}

### Task 3: Bob (Receiver)

In the next step of the protocol, Bob receives qubits and randomly selects bases to measure in. 

**Summary**

Measures each qubit in a randomly chosen basis, X or Z, and returns the chosen bases and the measurement results.

**Input**

qs: Array of qubits which were prepared by the sender

**Output**

A tuple of two integer arrays. Each element of the first array indicates the basis chosen for measurement, 0 for Z, 1 for X. The corresponding element of the second array indicates whether the qubit was measured as 0 or 1 in that basis.


In [5]:
%%qsharp
operation Bob(qs : Qubit[]) : (Int[], Int[]) {
    mutable measurements = [];
    mutable bases = [];
    for q in qs{
        if( RandomBit() == 1){
            let measurement = Measure ([PauliX], [q]) == Zero ? 0|1;
            set measurements += [measurement];
            set bases += [1];
        }
        else{
            let measurement = Measure ([PauliZ], [q]) == Zero ? 0|1;
            set measurements += [measurement];
            set bases += [0];
        }
    }
    return (bases, measurements);
}

### Task 4: Putting it all together

So now we just need to call our operations in the right order to perform key distribution. The length of generated keys is probabilistic, since we do not know how many times the sender and the receiver will choose the same basis. Our BB84 operation takes an argument $N$ which specifies the number of protocol iterations we run. The length of our key will be between 0 and N, and on average it will be $\frac{1}{2}N$.

**Summary**

This operation acts as the intermediary exchanging both classical and quantum information between parties.
$N$ controls the number of qubits transmitted, so our final key will be of length less than or equal to $N$.

**Input**

N: The number of qubits to attempt key distribution with. 

**Output**

A tuple of 5 integer arrays. 
- 1 - The preparation bases the sender used.
- 2 - The measurement bases the receiver used.
- 3 - The bits that the sender prepared in either basis.
- 4 - The bits that were measured by the receiver.
- 5 - The key that the receiver made after comparing bases.

In [6]:
%%qsharp
operation BB84(N : Int) : (Int[], Int[], Int[], Int[], Int[]) {
    //let result = [];
    use qubits = Qubit[N];
        
    //preparation bases
    let (prep_bases, avalues) = Alice(qubits);
    Message($"avalues: {avalues}");

    //measure
    let (meas_bases, bvalues) = Bob(qubits);
    Message($"bvalues: {bvalues}");

    //compare
    mutable key = new Int[0];
    for i in 0..(Length(meas_bases) - 1){
        if(prep_bases[i] == meas_bases[i]){
            set key += [avalues[i]];
        }
    }
    
    return (prep_bases, meas_bases, avalues, bvalues, key);
}

/snippet_.qs(14,19): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


### Check your work

In [7]:
def formatOutput(basesS, basesR, bitS, bitR, key):
    keyCopy = key.copy()
    same = "|" 
    basisSentChar = "|"
    basisRecChar = "|"
    bitSent = "|"
    bitRec = "|"
    keyS = "|"
    keyR = "|"
    stateSent = "|"
    for i in range(len(basesR)):
        bitSent += ' 1 |' if bitS[i] == 1 else ' 0 |'
        bitRec += ' 1 |' if bitR[i] == 1 else ' 0 |'
        same += ' y |' if basesR[i] == basesS[i] else ' n |'
        keyS += ' {} |'.format(bitS[i]) if basesR[i] == basesS[i] else "   |"
        keyR += ' {} |'.format(key.pop(0)) if basesR[i] == basesS[i] else "   |"
        basisRecChar += ' Z |' if basesR[i] == 0 else ' X |'
        if basesS[i] == 0:
            stateSent += "|0>|" if bitS[i] == 0 else "|1>|"
            basisSentChar += " Z |"
        else:
            stateSent += "|+>|" if bitS[i] == 0 else "|->|"
            basisSentChar += " X |"

    print("Let's compare this to the selected bases, and the transmitted qubit states")
    print("Alice's bases were:       {}".format(basisSentChar))
    print("Bob's bases were:         {}".format(basisRecChar))
    print("Both bases match (yes/no):{}".format(same))
    print("Alice's encoded bit:      {}".format(bitSent))
    print("The states sent were:     {}".format(stateSent))
    print("Bob measured:             {}".format(bitRec))
    print("Notice how the key is formed by the bits where bases are equal")
    print("Bob's key:                {}".format(keyR))
    print("Alice's key:              {}".format(keyS))
    print("The key that was generated was {}\n".format(keyCopy))
(basesS, basesR, bitS, bitR, key) = BB84.simulate(N=20)
formatOutput(basesS, basesR, bitS, bitR, key)

avalues: [1,0,0,0,0,1,1,0,1,1,0,1,1,0,1,1,0,0,1,1]
bvalues: [1,0,1,0,0,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1]
Let's compare this to the selected bases, and the transmitted qubit states
Alice's bases were:       | Z | Z | Z | X | X | X | Z | Z | Z | Z | X | X | Z | Z | Z | Z | X | Z | X | Z |
Bob's bases were:         | X | Z | X | Z | X | Z | X | Z | Z | Z | Z | Z | Z | X | Z | X | Z | X | X | Z |
Both bases match (yes/no):| n | y | n | n | y | n | n | y | y | y | n | n | y | n | y | n | n | n | y | y |
Alice's encoded bit:      | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | 1 |
The states sent were:     ||1>||0>||0>||+>||+>||->||1>||0>||1>||1>||+>||->||1>||0>||1>||1>||+>||0>||->||1>|
Bob measured:             | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 |
Notice how the key is formed by the bits where bases are equal
Bob's key:                |   | 0 |   |   | 0 |   |   | 0 | 1 | 1 |   |   | 1 |   | 1 |   |   |   | 1 | 1 |
Alic